In [1]:
import numpy as np
import pandas as pd
import warnings

# 1. 데이터 수집
## 1.1 종목 불러오기

In [2]:
def get_ticker(markets:list):
    import pandas as pd 
    
    lst_tickers = []
    for market in markets:
        df_ticker = pd.read_csv(f'tickers_by_markets/{market}.csv')
        tickers = df_ticker[df_ticker['Exchange'] == market]['Ticker'].tolist()
        lst_tickers.extend(tickers)
    return lst_tickers
lst_tickers = get_ticker(markets=['KSC', 'KOE']) # KSC: KOSPI, KOE: KOSDAQ
print(len(lst_tickers))
print(lst_tickers[:5])

1720
['001065.KS', '096760.KS', '001067.KS', '089590.KS', '005930.KS']


## 1.2 주가 데이터 불러오기¶

In [8]:
def read_data(date:list, tickers:list):
    from datetime import datetime
    import time
    from dateutil.relativedelta import relativedelta
    import pandas as pd
    import yfinance as yf
    
    lst_code = ' '.join(tickers)
    
    start = datetime.strptime(date[0], "%Y-%m-%d")
    end = datetime.strptime(date[1], "%Y-%m-%d") # string to datetime
    m0 = start
    m1 = start + relativedelta(months = 1)
    
    smonth= (end.year - start.year -1) * 12 + (end.month) + (12-start.month+1);smonth  # 총 month 수 구하기
    monthlist = [(m0,m1)]
    globals()['m{}'.format(smonth)] = end

    for i in range(1,smonth):
        globals()['m{}'.format(i)]= start + relativedelta(months=i)   # 변수를 총 month수만큼 생성해서 날짜 구하기
                                                                        # (m0 = ['2017-01-02'], ...,m47 = ['2020-12-02'])    
        (first,last) = (start + relativedelta(months=i),start + relativedelta(months=i+1))
        monthlist.append((first,last))
        
        
    ndf = pd.DataFrame()
    
    for (mn,mm) in monthlist: 

        df = yf.download(tickers=lst_code, start=mn, end=mm, group_by='tickers', progress=True).reset_index()
        # df 예쁘게 만들기
        df = df.melt(id_vars='Date') # Date로 녹이기
        df = df.pivot_table(index=['variable_0', 'Date'], columns='variable_1', values='value').reset_index()
        df = df.rename(columns={'variable_0':'Code'})
        df.columns.names= [None]
        df = df[['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

        # df를 다 concat해버리기
        ndf = pd.concat([ndf,df],ignore_index = True)

    ndf = ndf.sort_values(by=['Code','Date'], axis=0,ignore_index = True)
    
    return ndf

In [10]:
date = ['2016-01-02', '2020-12-31']

stock_data = read_data(date=date, tickers=lst_tickers)
stock_data.to_csv('C:\\Users\\chauj\\Desktop\\KISTI 공모전') #csv파일로 생성

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-10-9ebb7b229475>, line 4)

In [ ]:
stock_data2[stock_data2['Code'] == '530004.KS']

In [ ]:
date = ['2017-01-02', '2020-12-31']

stock_data2 = read_data(date=date, tickers=lst_tickers)
stock_data2

# 결론
- 안 불러지는 종목은 두 가지 이유에서이다. 하나는 입력한 기간(2017~2021)의 데이터가 전부 있지 않은 데이터가 제외된다.(예:상장된 지 5년 미만인 종목은 데이터가 5년 미만으로 있기 때문에 불러와지지 않음) 또 다른 이유는 상장 폐지 등의 이유로 종목 자체가 사라진 경우이다. 'symbol may be delisted'라고 에러가 뜨는 경우 거의 대부분 여기에 해당된다.

- 문제는 데이터를 불러올 때마다 에러가 나는 데이터의 개수, 종류가 달라진다는 점인데 이건 어차피 에러가 나지 않아도 위의 두 가지 문제가 있는 데이터는 데이터프레임에 공백으로 표시 되므로 사용자가 특정 종목을 검색해서 쓸 때는 문제가 되지 않는다.

- 만일 특정 시장(KOSPI, KOSDAQ 등)의 전체적인 데이터가 필요할 경우 이런 공백에 상관없이 해당 데이터를 로드하면 된다. 로드하는 방법은 
아래 코드를 입력하면 된다.

`
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

data = pdr.get_data_yahoo('^KS11',start = '2020-01-01', end = '2021-01-01')
print(data)
`

## KOSPI 데이터 로드하기

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

data = pdr.get_data_yahoo('^KS11',start = '2020-01-01', end = '2021-01-01')
print(data)

- 1개월 단위로 나눠서 총 5년치(변경가능) 데이터프레임 가져오기
- 데이터 빠진 건 없는지 다시 확인

여기서부터

In [4]:
from datetime import datetime
import time
from dateutil.relativedelta import relativedelta

date = ['2020-11-02', '2020-12-31']
#date = [start, end]

start = datetime.strptime(date[0], "%Y-%m-%d")
end = datetime.strptime(date[1], "%Y-%m-%d") # string to datetime

In [5]:
start

datetime.datetime(2020, 11, 2, 0, 0)

In [8]:
smonth= (end.year - start.year -1) * 12 + (end.month) + (12-start.month+1);smonth  # 총 month 수 구하기
m0 = start
m1 = start + relativedelta(months = 1)
monthlist = [(m0,m1)]

globals()['m{}'.format(smonth)] = end

for i in range(1,smonth):
    globals()['m{}'.format(i)]= start + relativedelta(months=i)   # 변수를 총 month수만큼 생성해서 날짜 구하기
                                                                    # (m0 = ['2017-01-02'], ...,m47 = ['2020-12-02'])    
    (first,last) = (start + relativedelta(months=i),start + relativedelta(months=i+1))
    monthlist.append((first,last))

In [9]:
monthlist

[(datetime.datetime(2020, 11, 2, 0, 0), datetime.datetime(2020, 12, 2, 0, 0)),
 (datetime.datetime(2020, 12, 2, 0, 0), datetime.datetime(2021, 1, 2, 0, 0))]

In [10]:
lst_tickers = get_ticker(markets=['KSC','KOE'])
tickers = lst_tickers

In [11]:
import pandas as pd
import yfinance as yf
lst_code = ' '.join(tickers)

ndf = pd.DataFrame()
# start, end 자리에 계속 변경
for (mn,mm) in monthlist: 

    df = yf.download(tickers=lst_code, start=mn, end=mm, group_by='tickers', progress=True).reset_index()

    df = df.melt(id_vars='Date') # Date로 녹이기
    df = df.pivot_table(index=['variable_0', 'Date'], columns='variable_1', values='value').reset_index()
    df = df.rename(columns={'variable_0':'Code'})
    df.columns.names= [None]
    df = df[['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

    # df를 다 concat해버리기
    ndf = pd.concat([ndf,df],ignore_index = True)
    
ndf = ndf.sort_values(by=['Code','Date'], axis=0,ignore_index = True)
ndf

[*********************100%***********************]  763 of 763 completed

132 Failed downloads:
- 083610.KS: No data found, symbol may be delisted
- 139200.KS: No data found, symbol may be delisted
- 530019.KS: No data found for this date range, symbol may be delisted
- 000327.KS: No data found, symbol may be delisted
- 570008.KS: No data found, symbol may be delisted
- 002270.KS: No data found, symbol may be delisted
- 550007.KS: No data found, symbol may be delisted
- 520005.KS: No data found for this date range, symbol may be delisted
- 500002.KS: No data found for this date range, symbol may be delisted
- 013367.KS: No data found, symbol may be delisted
- 083390.KS: No data found, symbol may be delisted
- 500016.KS: No data found for this date range, symbol may be delisted
- 068875.KS: No data found for this date range, symbol may be delisted
- 530010.KS: No data found, symbol may be delisted
- 530008.KS: No data found, symbol may be delisted
- 75A511.KS: No data found, symbol may 

,Code,Date,Open,High,Low,Close,Adj Close,Volume
0,000040.KS,2020-11-02,971.0,1015.0,952.0,968.0,968.0,2552524.0
1,000040.KS,2020-11-03,970.0,1010.0,969.0,1005.0,1005.0,1914498.0
2,000040.KS,2020-11-04,1015.0,1020.0,970.0,981.0,981.0,1788193.0
3,000040.KS,2020-11-05,990.0,998.0,975.0,976.0,976.0,1418743.0
4,000040.KS,2020-11-06,978.0,985.0,942.0,969.0,969.0,2000424.0
...,...,...,...,...,...,...,...,...
26497,900140.KS,2020-12-23,3550.0,3610.0,3490.0,3555.0,3555.0,790508.0
26498,900140.KS,2020-12-24,3600.0,3785.0,3485.0,3610.0,3610.0,2197863.0
26499,900140.KS,2020-12-28,3575.0,3680.0,3530.0,3560.0,3560.0,879661.0
26500,900140.KS,2020-12-29,3680.0,3790.0,3645.0,3790.0,3790.0,1095775.0


In [ ]:
# 두 데이터프레임 비교하기, 차이나는 열 출력하는 코드

df = pd.concat([df1,df2])
df = df.reset_index(drop=True) # 인덱스 초기화
 
df_grp = df.groupby(df.columns.tolist()) # 전체 열 비교
df_di = df_grp.groups # 딕셔너리로 만들기 
 
idx = [x[0] for x in df_di.values() if len(x) == 1] # 인덱스 검토
df.loc[idx,:] # Same as df.reindex(idx)

1. start에 한달 더하기(d1)
1-1. [start,d1]로 데이터 다운로드
2. d1에 한달 더하기(d2)
2-1. [d1, d2]로 데이터 다운로드
3. d2에 한달 더하기(d3)
3-1. [d2,d3]로 데이터 다운로드
4. d_(n-2)에서 한달 더하기 (d_(n-1)(before end) 만들기)
4-1. [d_(n-2), d(n_1)] 로 데이터 다운로드
5. 
5-1. end에서 하나 적은 d_(n-1)에서 end까지 데이터 다운로드



실험1. 2020-12-30 기준 코스닥 상장 종목 수는 1,582, 기업 수는 1,578, 근데 yfinance에는 957종목 뿐
실험2.  2020-12-30 기준 코스닥 상장 종목 수는 939, 기업 수는 822, 근데 yfinance에는 763종목 뿐
모든 해에 종목 수가 일정함. 실제론 아님-> 상장 종목 업데이트가 안되나?
근데 상장된지 1개월밖에 안된 종목이 있었는데......뭐지??

한국 시장 데이터가 많이 누락되었다는 말은 여기서 나온 말 같음

실험3. 두 번 실행했을 때 에러나는 종목의 개수와 종류가 달랐지만 결과인 데이터프레임은 완벽히 일치한다.


문제1. 1개월 단위로 불러오면 1개월치 데이터가 31일 full로 존재하지 않는 이상 버려진다.
해결방법1. 일괄 상장반영일이 있다면 찾아보고, 없다면 며칠 단위로 쪼개는게 best인지 실험해봐야함.

문제2. 에러랑 진행바가 의미가 없다. 찾으려는 개월 수를 100으로 하는 바를 새로 만들 순 없나?

KSC, KOE

2016-2021